<a href="https://colab.research.google.com/github/lorenzrossi/InfoRet_SemanticSimilarity/blob/main/InfoRet_Semantic_Similarity_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Generics

!pip install wget
!pip install opendatasets

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import shutil, sys, tqdm, re, spacy, string, argparse, os, pickle, pprint, gc, io, pdb, zipfile, wget, h5py, subprocess

from datetime import datetime
import opendatasets as op

from gensim.models import KeyedVectors
from string import punctuation

# metrics 
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

from keras.preprocessing import sequence

# PYTORCH
import torch
import torch.nn as nn
import torchtext
from torchtext import datasets
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence 
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim
import random

# NLTK
import nltk as nlp
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from collections import Counter
nlp.download('punkt')
nlp.download('wordnet')
nlp.download('stopwords')
nlp.download('popular')
stop_words = stopwords.words('english')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=b58fdb4ffbad5dd7ca61cdd6fff89bf0eb6956ce1b4f4f216338aa7204c74aae
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk

In [2]:
def download_data(glove='glove.6B.zip'):
    print("Downloading word embedding")
    downloaded_glove1 = wget.download("http://nlp.stanford.edu/data/{}".format('glove.6B.zip'))
    
    if not os.path.exists("./data"):
        os.mkdir("./data")
    print("Extracting")
    zip = zipfile.ZipFile(downloaded_glove1)
    zip.extractall(path="./data")
    print("done!")

In [3]:
download_data()

Extracting
done!


In [4]:
#!wget https://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove.6B.zip

In [5]:
glove_path = './data/glove.6B.300d.txt'

In [7]:
op.download("https://www.kaggle.com/datasets/stanfordu/stanford-natural-language-inference-corpus")

dataset_folder = 'stanford-natural-language-inference-corpus'

# bcc30972b8b9f25c2bc6c0a46d8f4d62

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: lorenzrossi
Your Kaggle Key: ··········


100%|██████████| 44.4M/44.4M [00:00<00:00, 90.4MB/s]


In [9]:
train_df = pd.read_csv(os.path.join(dataset_folder, "snli_1.0_train.csv"))
val_df = pd.read_csv(os.path.join(dataset_folder, "snli_1.0_dev.csv"))

train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]

train_df = train_df[['gold_label', 'sentence1', 'sentence2']][train_df['gold_label'] != '-']
val_df = val_df[['gold_label', 'sentence1', 'sentence2']][val_df['gold_label'] != '-']

In [10]:
def get_word_map(count):  
  word_map = {}
  for num in count:
    if num in word_map:
      word_map[num] += 1
    else:
      word_map[num] = 1

  return word_map

In [11]:
count_premise = train_df['sentence1'].str.split().str.len()
get_word_map(count_premise)

{11: 47950,
 6: 25521,
 13: 37865,
 26: 3338,
 7: 36562,
 14: 32913,
 10: 50609,
 15: 26970,
 17: 19876,
 23: 6481,
 16: 22773,
 22: 8132,
 35: 510,
 18: 18052,
 9: 49129,
 12: 43021,
 8: 43177,
 5: 10811,
 19: 15722,
 21: 10004,
 42: 135,
 24: 5250,
 33: 786,
 20: 12986,
 4: 3314,
 25: 4218,
 30: 1259,
 31: 1208,
 36: 384,
 37: 328,
 27: 2528,
 3: 639,
 29: 1938,
 28: 1946,
 51: 33,
 32: 797,
 38: 322,
 2: 63,
 39: 207,
 50: 51,
 34: 723,
 40: 170,
 43: 60,
 53: 15,
 48: 42,
 49: 48,
 41: 123,
 55: 15,
 78: 15,
 45: 24,
 44: 99,
 54: 9,
 62: 6,
 46: 57,
 61: 3,
 63: 6,
 57: 3,
 64: 30,
 47: 48,
 59: 3,
 70: 3,
 52: 3,
 60: 6,
 56: 3,
 58: 3,
 68: 15,
 65: 6,
 69: 15,
 72: 3,
 73: 3}

In [12]:
count_hypothesis = train_df['sentence2'].str.split().str.len()
get_word_map(count_hypothesis)

{9: 46982,
 7: 88656,
 6: 93996,
 4: 62090,
 16: 3637,
 5: 70582,
 8: 61890,
 12: 15940,
 14: 7374,
 17: 2448,
 11: 22992,
 20: 839,
 19: 1209,
 10: 33588,
 2: 1609,
 13: 10868,
 18: 1748,
 3: 15735,
 21: 565,
 15: 5184,
 23: 288,
 24: 173,
 27: 66,
 33: 14,
 25: 150,
 31: 22,
 22: 398,
 30: 28,
 32: 12,
 35: 10,
 28: 59,
 26: 103,
 29: 38,
 38: 6,
 1: 42,
 43: 2,
 34: 9,
 40: 3,
 48: 1,
 36: 4,
 56: 1,
 37: 2,
 50: 1,
 46: 1,
 55: 1,
 39: 1}

In [13]:
def clean_text(text, stop_words = False, lemmatization = False):
  text = text.lower().split()
  if stop_words:
    stop = stopwords.words('english')
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
  
  text = " ".join(text)
  text = re.sub("[^A-Za-z']+", ' ', str(text)).replace("'", '')
  text = re.sub(r"\bum*\b", "", text)
  text = re.sub(r"\buh*\b", "", text)
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  if lemmatization:
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])
  
  text = text.translate(str.maketrans('', '', punctuation))
  return text.strip()

In [14]:
train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [15]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]

In [16]:
df = pd.concat([train_df, val_df])

# Tokenization and Vocabulary

In [17]:
def pair_generator(df):
  sentence_pair  = []
  sentence_label = []
  for _, row in df.iterrows():
    sentence_pair.append((row['sentence1'], row['sentence2']))
    sentence_label.append(row['gold_label'])
  return sentence_pair, sentence_label

In [18]:
sentence_pairs, _ = pair_generator(df)
train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

In [19]:
labels = set(train_sentence_labels)
print(labels)

{'contradiction', 'entailment', 'neutral'}


In [20]:
tag2idx = {word: i for i, word in enumerate(labels)}
print(tag2idx)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}


In [21]:
train_labels = [tag2idx[t] for t in train_sentence_labels]
val_labels = [tag2idx[t] for t in val_sentence_labels]

In [22]:
class Vocabulary:
  def __init__(self):
    self.word2index = {}
    self.word2count = {}
    self.index2word = {}
    self.n_words = 0

  def addSentence(self, sentence):
    for word in word_tokenize(sentence):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words + 1
      self.word2count[word] = 1
      self.index2word[self.n_words + 1] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [23]:
class DataSetLoader(Dataset):
  def __init__(self, sentence_pair, labels):
    self.sentence_pair = sentence_pair
    self.labels        = labels

  def __len__(self):
    return len(self.sentence_pair)

  def __getitem__(self, index):
    return self.sentence_pair[index], self.labels[index]

In [24]:
vocab = Vocabulary()
for data in [sentence_pairs]:
  for sentence_pair in data:
    premise = sentence_pair[0]
    hypothesis = sentence_pair[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

print("Vocab size:", len(vocab.word2index))

Vocab size: 33353


In [25]:
def get_pair_indices(sentence_pairs):
  indices_pairs = []
  for sentence_pair in sentence_pairs:
    premise = sentence_pair[0]
    premise_indices = [vocab.word2index[w] for w in word_tokenize(premise)]
    hypothesis = sentence_pair[1]
    hypothesis_indices = [vocab.word2index[w] for w in word_tokenize(hypothesis)]
    indices_pairs.append((premise_indices, hypothesis_indices))
  return indices_pairs

In [26]:
train_data = DataSetLoader(get_pair_indices(train_sentence_pairs), train_labels)
val_data = DataSetLoader(get_pair_indices(val_sentence_pairs), val_labels)

# Word Embedding

In [28]:
EPOCHS = 10
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 64
LEARNING_RATE = 0.001
STACKED_LAYERS = 2

In [29]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

17168 308


In [30]:
def get_coefs(word, *arr):
  return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
  with open(path) as f:
    return dict(get_coefs(*line.strip().split(' ')) for line in f)

embeddings_index = load_embeddings(glove_path)

In [31]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

Embedded count: 27188


(33354, 300)

# LSTM

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [33]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
class LSTM(nn.Module):
  def __init__(self, vocab_size, hidden_size, target_size, stacked_layers, weights_matrix, bidirectional):
    super(LSTM, self).__init__()
    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.bidirectional = bidirectional
    self.target_size = target_size
    self.stacked_layers = stacked_layers
    num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
    self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    self.embedding.weight.data.copy_(torch.from_numpy(weights_matrix))
    self.embedding.weight.requires_grad = True

    self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size, num_layers=self.stacked_layers, 
                        batch_first = True, dropout=0.2, bidirectional=bidirectional)
    
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p = 0.2)

    self.FC_concat1 = nn.Linear(2 * 2 * hidden_size if bidirectional else 2 * hidden_size, 128)
    self.FC_concat2 = nn.Linear(128, 64)
    self.FC_concat3 = nn.Linear(64, 32)

    for lin in [self.FC_concat1, self.FC_concat2]:
      nn.init.xavier_uniform_(lin.weight)
      nn.init.zeros_(lin.bias)

    self.output = nn.Linear(32, self.target_size)

    self.out = nn.Sequential(
			self.FC_concat1,
			self.relu,
			self.dropout,
			self.FC_concat2,
			self.relu,
      self.FC_concat3,
      self.relu,
			self.dropout,
			self.output
		)

  def forward_once(self, seq, hidden, seq_len):
    embedd_seq = self.embedding(seq)
    packed_seq = pack_padded_sequence(embedd_seq, lengths=seq_len, batch_first=True, enforce_sorted=False)
    output, (hidden, _) = self.lstm(packed_seq, hidden)
    return hidden

  def forward(self, input, premise_len, hypothesis_len):
    premise    = input[0]
    hypothesis = input[1]
    batch_size = premise.size(0)

    h0 = torch.zeros(self.stacked_layers*2 if self.bidirectional else self.stacked_layers, batch_size, self.hidden_size).to(device) # 2 for bidirection 
    c0 = torch.zeros(self.stacked_layers*2 if self.bidirectional else self.stacked_layers, batch_size, self.hidden_size).to(device)

    # hidden = self.init_hidden(batch_size)

    premise    = self.forward_once(premise, (h0, c0), premise_len)
    hypothesis = self.forward_once(hypothesis, (h0, c0), hypothesis_len)
    
    combined_outputs  = torch.cat((premise, hypothesis, torch.abs(premise - hypothesis), premise * hypothesis), dim=2)

    return self.out(combined_outputs[-1])

In [35]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [36]:
def train(model, train_loader, val_loader, criterion, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for val in train_loader:
      sentence_pairs, labels = map(list, zip(*val))

      premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in sentence_pairs]
      hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in sentence_pairs]
      batch = len(premise_seq)

      premise_len    = list(map(len, premise_seq))
      hypothesis_len = list(map(len, hypothesis_seq))

      temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
      premise_seq    = temp[:batch, :]
      hypothesis_seq = temp[batch:, :]
      labels         = torch.tensor(labels).long().to(device)

      model.zero_grad()
      prediction = model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)

      loss = criterion(prediction, labels)
      acc  = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for val in val_loader:
        sentence_pairs, labels = map(list, zip(*val))

        premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in sentence_pairs]
        hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in sentence_pairs]
        batch = len(premise_seq)

        premise_len    = list(map(len, premise_seq))
        hypothesis_len = list(map(len, hypothesis_seq))

        temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
        premise_seq    = temp[:batch, :]
        hypothesis_seq = temp[batch:, :]

        premise_seq    = premise_seq.to(device)
        hypothesis_seq = hypothesis_seq.to(device)
        labels         = torch.tensor(labels).long().to(device)

        model.zero_grad()
        prediction = model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
        
        loss = criterion(prediction, labels)
        acc  = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    torch.cuda.empty_cache()

In [38]:
lstm_model = LSTM(vocab_size = VOCAB_SIZE, 
                  hidden_size = HIDDEN_SIZE, 
                  target_size = TARGET_SIZE, 
                  stacked_layers = STACKED_LAYERS, 
                  weights_matrix = weights, 
                  bidirectional = True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

LSTM(
  (embedding): Embedding(33354, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [39]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

00:04:18.42
Epoch 1: train_loss: 0.6911 train_acc: 0.7096 | val_loss: 0.5801 val_acc: 0.7603
00:04:18.42
Epoch 2: train_loss: 0.5851 train_acc: 0.7651 | val_loss: 0.5226 val_acc: 0.7885
00:04:18.42
Epoch 3: train_loss: 0.5486 train_acc: 0.7828 | val_loss: 0.5075 val_acc: 0.8002
00:04:18.89
Epoch 4: train_loss: 0.5256 train_acc: 0.7936 | val_loss: 0.5006 val_acc: 0.7991
00:04:17.69
Epoch 5: train_loss: 0.5084 train_acc: 0.8014 | val_loss: 0.4852 val_acc: 0.8110
00:04:17.79
Epoch 6: train_loss: 0.4954 train_acc: 0.8075 | val_loss: 0.4808 val_acc: 0.8087
00:04:17.94
Epoch 7: train_loss: 0.4850 train_acc: 0.8121 | val_loss: 0.4733 val_acc: 0.8118
00:04:18.56
Epoch 8: train_loss: 0.4771 train_acc: 0.8158 | val_loss: 0.4730 val_acc: 0.8131
00:04:19.60
Epoch 9: train_loss: 0.4680 train_acc: 0.8203 | val_loss: 0.4777 val_acc: 0.8161
00:04:17.81
Epoch 10: train_loss: 0.4623 train_acc: 0.8223 | val_loss: 0.4770 val_acc: 0.8157
